In [1]:

from sklearn.metrics import r2_score
import copy
import operator
from tensorflow.keras.losses import BinaryCrossentropy
import random
import pandas as pd
import keras.metrics as metrics
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sys

import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Flatten,Dense,Input,Reshape
from tensorflow.keras.models import Sequential
from random import randrange



In [2]:
from genetic_algorithm_net import GA

SyntaxError: invalid syntax (genetic_algorithm_net.py, line 112)

### Import dataset

In [ ]:
df = pd.read_csv('../Data/processed_df.csv')

In [ ]:
df.head()

,handgrip,weight_loss,exaustion,poor_endurance,gait_speed,balance,hearing_deficit,distance_vision,near_vision,depression_scale,...,idade_morador_8,idade_morador_9,idade_morador_10,idade_morador_11,idade_morador_12,idade_morador_13,idade_morador_14,adl_index,aadl_index,iadl_index
0,24.333333,0.0,1.0,1.0,3.500,4.0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0
1,25.666667,0.0,1.0,1.0,3.300,4.0,4.0,3.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0,8.0
2,14.666667,NaN,2.0,1.0,4.595,4.0,2.0,3.0,4.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0
3,24.000000,0.0,1.0,1.0,5.610,4.0,3.0,2.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,5.0,8.0
4,NaN,0.0,4.0,2.0,38.145,NaN,3.0,2.0,3.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,3.0,3.0


In [ ]:
df.shape

(9412, 63)

### Remove missing data

In [ ]:
ci_map = {
    'cognitive': ['semantic_memory', 'verbal_fluency', 'memory_recall', 'temporal_orientation'],
    'psychologicao': ['depression_scale', 'sleep_quality'],
    'locomotor': ['gait_speed', 'balance'],
    'vitality': ['handgrip', 'poor_endurance', 'weight_loss', 'exaustion'],
    'sensory': ['distance_vision', 'near_vision', 'hearing_deficit']
}

ci_cols = []
[ci_cols.extend(value) for value in ci_map.values()]
ci_cols = ci_cols + ['iadl_index']
ci_cols

['semantic_memory',
 'verbal_fluency',
 'memory_recall',
 'temporal_orientation',
 'depression_scale',
 'sleep_quality',
 'gait_speed',
 'balance',
 'handgrip',
 'poor_endurance',
 'weight_loss',
 'exaustion',
 'distance_vision',
 'near_vision',
 'hearing_deficit',
 'iadl_index']

In [ ]:
df_clean = df.dropna(subset=ci_cols)
df_clean.shape

(6916, 63)

In [ ]:
df_clean['iadl_index'] = df_clean['iadl_index'].map(lambda x: 0 if x == 8 else 1)

C:\Users\mwuillau\AppData\Local\Temp\ipykernel_22968\2683086268.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['iadl_index'] = df_clean['iadl_index'].map(lambda x: 0 if x == 8 else 1)


### Split Train and test


In [ ]:
import pickle
# Define dataset
df_autoencoder = df_clean[ci_cols]
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_autoencoder.drop(columns=['iadl_index']), df_autoencoder['iadl_index'], test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [ ]:

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

X_train_scaled.shape, X_val_scaled.shape, X_test_scaled.shape


with open('./out/autoencoder_scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)


In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_train_scaled_over, y_train_over = ros.fit_resample(X_train_scaled, y_train)

#### Run GA

In [ ]:
DNA_parameter = [["tanh","softmax","relu","sigmoid","linear"],
                 ["sgd","adam"]]



In [ ]:
ga = GA(coding_size=2, X_train=X_train_scaled_over, X_test=X_val_scaled, y_train=y_train_over, y_test=y_val, DNA_parameter=DNA_parameter, epochs=50)

In [ ]:
pop = ga.create_population(hidden_layers_encoder=3, hidden_layers_decoder=3, max_dim_increase=30, population_size=20)

In [ ]:
best_arch = ga.GA(population=pop, n_generations=100)

Generation: 0


44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7769 - f1_score: 0.2923 - loss: 0.1156 - precision: 0.3900 - recall: 0.5454
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7886 - f1_score: 0.2923 - loss: 0.1151 - precision: 0.3998 - recall: 0.4871     
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8025 - f1_score: 0.2923 - loss: 0.1257 - precision: 0.4257 - recall: 0.4389
Best: 
------------------------------
15 None
14 tanh
4 sigmoid
2 relu
1 sigmoid
optimizer sgd
------------------------------
Second Best: 
------------------------------
15 None
12 linear
4 tanh
2 linear
1 sigmoid
optimizer adam
------------------------------
Loser: 
------------------------------
15 None
15 sigmoid
8 sigmoid
4 tanh
2 linear
1 sigmoid
optimizer sgd
------------------------------
Best Child: 
------------------------------
15 None
14 tanh
12 linear
4 sigmoid
4 tanh
2 relu
1 sigmoid
optimizer sgd
------------------------------
Second Best Child: 
------------------------------
15 None


KeyboardInterrupt: 

### Import Genetic Result

In [ ]:
model = tf.keras.models.load_model('./out/best_model.h5')

In [ ]:
model.predict(X_test_scaled)

32/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 

44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step


array([[-0.2995344 , -0.41704544, -0.3759935 , ...,  0.19974044,
         0.05309695,  0.05122371],
       [-0.54145867, -0.43984187, -0.53072876, ...,  0.88966703,
         0.8229447 ,  0.57867724],
       [-0.38256487, -0.39333206, -0.35648096, ...,  0.77078557,
         0.67072767,  0.5331141 ],
       ...,
       [ 0.73997784,  0.7839768 ,  0.79190093, ..., -0.09436029,
         0.05686268, -0.16890971],
       [-0.1041927 , -0.20652618, -0.01900695, ..., -0.10671389,
        -0.18163204,  0.04037876],
       [-0.05616935, -0.08047217, -0.03674935, ..., -0.20170137,
        -0.27207446, -0.21064   ]], dtype=float32)

In [ ]:
from sklearn.metrics import r2_score

decoded_data = model.predict(X_test_scaled)
global_r2 = r2_score(X_test_scaled, decoded_data)
print(global_r2)

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
0.38231258232286514
